In [30]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path

In [32]:
%%time

AID=825
READ_FROM_FILE=True
DATA_DIR = 'data'
if READ_FROM_FILE:
    active_fname=DATA_DIR + '/AID%s_active.sdf'%AID
    inactive_fname=DATA_DIR + '/AID%s_inactive.sdf'%AID
else:
    active_fname, inactive_fname = get_assay(AID)


CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 21.9 µs


In [33]:
%%time

def similarity(x):
    return 1./1+x

iterable_pos=obabel.obabel_to_eden3d(active_fname, similarity, k = 20)
iterable_neg=obabel.obabel_to_eden3d(inactive_fname, similarity, k = 20)

from eden.graph import Vectorizer
vectorizer = Vectorizer(complexity=3, discretization_size = 5, discretization_dimension = 5)

Xp = vectorizer.fit_transform(iterable_pos, n_jobs = 1)
Xn = vectorizer.transform(iterable_neg, n_jobs = 1)

import numpy as np
from scipy.sparse import vstack

yp = [1] * Xp.shape[0]
yn = [-1] * Xn.shape[0]
y = np.array(yp + yn)
X = vstack([Xp, Xn], format="csr")


from sklearn import cross_validation
from sklearn.linear_model import SGDClassifier
estimator = SGDClassifier(class_weight='auto', shuffle=True)

for scoring in ['accuracy', 'precision', 'recall', 'f1', 'average_precision', 'roc_auc']:
    scores = cross_validation.cross_val_score(estimator, X, y, cv=5, scoring=scoring, n_jobs=-1)
    print('%20s: %.3f +- %.3f' % (scoring, np.mean(scores), np.std(scores)))

            accuracy: 0.644 +- 0.132
           precision: 0.742 +- 0.196
              recall: 0.504 +- 0.199
                  f1: 0.572 +- 0.139
   average_precision: 0.751 +- 0.182
             roc_auc: 0.709 +- 0.168
CPU times: user 42.9 s, sys: 814 ms, total: 43.7 s
Wall time: 45.1 s


In [34]:
%%time
iterable_pos=obabel.obabel_to_eden(active_fname)
iterable_neg=obabel.obabel_to_eden(inactive_fname)
from eden.graph import Vectorizer

vectorizer = Vectorizer(complexity=3)
Xp = vectorizer.transform(iterable_pos, n_jobs = 1)
Xn = vectorizer.transform(iterable_neg, n_jobs = 1)

import numpy as np
from scipy.sparse import vstack

yp = [1] * Xp.shape[0]
yn = [-1] * Xn.shape[0]
y = np.array(yp + yn)
X = vstack([Xp, Xn], format="csr")


from sklearn import cross_validation
from sklearn.linear_model import SGDClassifier
estimator = SGDClassifier(class_weight='auto', shuffle=True)

for scoring in ['accuracy', 'precision', 'recall', 'f1', 'average_precision', 'roc_auc']:
    scores = cross_validation.cross_val_score(estimator, X, y, cv=5, scoring=scoring, n_jobs=-1)
    print('%20s: %.3f +- %.3f' % (scoring, np.mean(scores), np.std(scores)))

            accuracy: 0.633 +- 0.115
           precision: 0.762 +- 0.184
              recall: 0.373 +- 0.221
                  f1: 0.466 +- 0.221
   average_precision: 0.617 +- 0.187
             roc_auc: 0.570 +- 0.190
CPU times: user 3.49 s, sys: 353 ms, total: 3.85 s
Wall time: 5.44 s


In [35]:
%matplotlib inline
from eden.util.display import draw_graph
from itertools import islice

for G in islice(iterable_pos, 3):
    draw_graph(G, vertex_label="atom_type")